### INSERT CUSTOMER DETAILS

In [0]:
%sql
CREATE OR REPLACE FUNCTION datalink.lineagedemo.insert_customer_details(
    customer_name STRING,
    phone_number STRING,
    email STRING
)
RETURNS STRING
LANGUAGE PYTHON
AS $$

"""
Inserts a new customer record into the datalink.lineagedemo.customer table.

Parameters:
    customer_name: Name of the customer.
    phone_number: Contact number of the customer.
    email: Email address of the customer.

Returns:
    A string indicating success with customer ID, or failure with error details.
"""
import requests
import json
import uuid
import re

# Configuration for Databricks SQL API
url = "" -- databricks_domain_url/api/2.0/sql/statements
warehouse_id = "" -- warehouse ID
bearer_token = "" -- bearer token

# Generate a unique customer ID
customer_id = str(uuid.uuid4())

# Normalize phone number
phone_number = re.sub(r'\D', '', phone_number)

# SQL statement to insert customer data
statement = f"""
INSERT INTO datalink.lineagedemo.customer(
    customer_id, customer_name, phone_number, email
)
VALUES (
    '{customer_id}', '{customer_name}', '{phone_number}', '{email}'
)
"""
# Set HTTP headers for the API request
headers = {
    "Authorization": f"Bearer {bearer_token}",
    "Content-Type": "application/json"
}
# Prepare the payload with SQL statement and settings
payload = {
    "statement": statement,
    "wait_timeout": "50s",
    "warehouse_id": warehouse_id
}
# Make the POST request to Databricks SQL API
response = requests.post(url, headers=headers, json=payload)
# Return success or error message based on response
if response.status_code == 200:
    return f"Customer created. ID: {customer_id}"
else:
    return f"Failed: {response.status_code} - {response.text}"
$$;

### INSERT VEHICLE DETAILS

In [0]:
%sql
CREATE OR REPLACE FUNCTION datalink.lineagedemo.insert_vehicle_details(
    cutomer_id STRING,
    model STRING,
    plate_number STRING,
    year STRING
)
RETURNS STRING
LANGUAGE PYTHON
AS $$
"""
Inserts vehicle information for a customer into the datalink.lineagedemo.vehicle_info table.

Parameters:
    customer_id: Unique identifier of the customer.
    model: Vehicle model (e.g., Toyota Corolla).
    plate_number: License plate number of the vehicle.
    year: Year of manufacture (e.g., 2022).

Returns:
    A string indicating success with vehicle ID, or failure with error details.
"""

import requests
import json
import uuid

# Static configuration - Databricks SQL endpoint details
url = "" -- databricks_domain_url/api/2.0/sql/statements
warehouse_id = "" -- warehouse ID
bearer_token = "" -- bearer token

# Generate a new unique vehicle ID
vehicle_id = str(uuid.uuid4())


# Prepare SQL statement to insert the vehicle record
statement = f"""
INSERT INTO datalink.lineagedemo.vehicle_info(
    vehicle_id, customer_id, model, plate_number, year
)
VALUES (
    '{vehicle_id}', '{cutomer_id}', '{model}', '{plate_number}', '{year}'
)
"""
# Set the headers for the API request
headers = {
    "Authorization": f"Bearer {bearer_token}",
    "Content-Type": "application/json"
}

# Create the request payload
payload = {
    "statement": statement,
    "wait_timeout": "50s",
    "warehouse_id": warehouse_id
}
# Send the SQL execution request
response = requests.post(url, headers=headers, json=payload)
# Handle response
if response.status_code == 200:
    return f"Vehicle info created. ID: {vehicle_id}"
else:
    return f"Failed: {response.status_code} - {response.text}"
$$;



### GET BOOKING STATUS INFO

In [0]:
%sql
CREATE OR REPLACE FUNCTION  datalink.lineagedemo.get_booking_status_info(vehicle_id STRING)
RETURNS TABLE(
    booking_id STRING,
    center_id STRING,
    center_name STRING,
    slot_id STRING,
    slot_datetime TIMESTAMP,
    service_name STRING,
    status STRING
)
RETURN (
    /*
        Retrieves booking status details for the given vehicle ID.
        Returns booking info including center details, slot timing, service name, and current status.
    */
    SELECT 
        bsi.booking_id,
        bsi.center_id,
        sci.location,                /* Service center location */
        bsi.slot_id,
        aslot.slot_datetime,        /* Date and time of the appointment slot */        
        st.service_name,            /* Name of the service booked */
        bsi.status                  /* Current status of the booking */
    FROM 
        datalink.lineagedemo.booking_status_info bsi
    JOIN 
        datalink.lineagedemo.appointment_slots aslot 
        ON bsi.slot_id = aslot.slot_id
    JOIN 
        datalink.lineagedemo.service_type st 
        ON bsi.service_type_id = st.service_type_id
    JOIN 
        datalink.lineagedemo.service_center_info sci 
        ON bsi.center_id = sci.center_id
    WHERE 
        bsi.vehicle_id = get_booking_status_info.vehicle_id
)



### GET AVAILABLE SLOTS

In [0]:
%sql
CREATE OR REPLACE FUNCTION  datalink.lineagedemo.get_available_slots(center_name STRING)
RETURNS TABLE(slot_id STRING,center_id STRING, location STRING, slot_datetime TIMESTAMP, is_available BOOLEAN) 
RETURN 
  (
  /*
      Retrieves all upcoming available slots that match the given center name or location.
      Filters out slots that are unavailable or scheduled in the past.
  */
  SELECT 
    slot_id,
    b.center_id, 
    b.location,                             /* Location of the service center */
    a.slot_datetime,                        /* Date and time of the available slot */
    a.is_available                          /* Indicates if the slot is currently available */
  FROM 
    datalink.lineagedemo.appointment_slots a 
  JOIN 
    datalink.lineagedemo.service_center_info b 
    ON a.center_id = b.center_id
  WHERE 
  (
    lower(b.location) LIKE '%' || lower(get_available_slots.center_name) || '%' 
    OR lower(b.center_name) LIKE '%' || lower(get_available_slots.center_name) || '%'
  )
  AND a.is_available = true                -- Only show slots that are available
  and slot_datetime>=current_timestamp()   -- Only include future slots
  ORDER BY 
    a.slot_datetime                        -- Sort by earliest slot first
)




### IS_SLOT_UPDATE_ALLOWED

In [0]:
%sql
CREATE OR REPLACE FUNCTION datalink.lineagedemo.is_slot_update_allowed(booking_id STRING)
RETURNS BOOLEAN
RETURN
    (
    /*
        Checks whether the slot for a given booking can still be updated.
        Allows updates only if the appointment is more than 48 hours from the current time.
    */
        SELECT CASE 
            WHEN MIN(slot_datetime) - CURRENT_TIMESTAMP > INTERVAL 48 HOURS THEN TRUE           -- Update allowed
            ELSE FALSE                                                                          -- Update not allowed
        END
        FROM datalink.lineagedemo.appointment_slots a
        JOIN datalink.lineagedemo.booking_status_info b
        ON a.slot_id = b.slot_id
        WHERE b.booking_id = is_slot_update_allowed.booking_id
    );


### GET_AVAILABLE_SERVICE

In [0]:
%sql
CREATE OR REPLACE FUNCTION  datalink.lineagedemo.get_available_service()
RETURNS TABLE(service_type_id STRING, service_name STRING, description STRING)
RETURN (
  /*
    Retrieves all available service types including their IDs, names, and descriptions.
    This function returns the complete list without any filters.
  */
  SELECT *
  FROM datalink.lineagedemo.service_type
);

### PROCESS_BOOKING_STATUS

In [0]:
%sql
CREATE OR REPLACE FUNCTION datalink.lineagedemo.process_booking_status(
    customer_id STRING,
    vehicle_id STRING,
    center_id STRING,
    slot_id STRING,
    service_type_id STRING,
    old_slot_id STRING,
    operation_type STRING
)
RETURNS STRING
LANGUAGE PYTHON
AS $$
"""
This function processes a booking operation (INSERT, UPDATE, DELETE) in the booking_status_info table.

- INSERT: Creates a new booking with status 'CONFIRMED' and updates the slot availability.
- UPDATE: Updates an existing booking's center, slot, and service info, and adjusts slot availability if changed.
- DELETE: Removes a booking and marks its slot as available.

Parameters:
    customer_id: ID of the customer
    vehicle_id: ID of the vehicle
    center_id: ID of the service center
    slot_id: ID of the new appointment slot
    service_type_id: ID of the service type
    old_slot_id: ID of the previously booked slot (used for update/delete)
    operation_type: Type of operation - INSERT, UPDATE, or DELETE

Returns:
    A status message string indicating success or failure.
"""
import requests
import json
import uuid

# Static configuration
url = "" -- databricks_domain_url/api/2.0/sql/statements
warehouse_id = "" -- warehouse ID
bearer_token = "" -- bearer token

# Generate booking ID for INSERT operations
booking_id = str(uuid.uuid4()) if operation_type == "INSERT" else ""

# Construct main SQL statement based on operation type
if operation_type == "INSERT":
    # Insert a new booking with CONFIRMED status
    statement = f"""
    INSERT INTO datalink.lineagedemo.booking_status_info (
        booking_id, customer_id, vehicle_id, center_id, slot_id, service_type_id, status
    )
    VALUES (
        '{booking_id}', '{customer_id}', '{vehicle_id}', '{center_id}', '{slot_id}', '{service_type_id}', 'CONFIRMED'
    );
    """
elif operation_type == "UPDATE":
    # Update the booking details
    statement = f"""
    UPDATE datalink.lineagedemo.booking_status_info
    SET status = 'CONFIRMED',
        center_id = '{center_id}',
        slot_id = '{slot_id}',
        service_type_id = '{service_type_id}'
    WHERE customer_id = '{customer_id}' AND vehicle_id = '{vehicle_id}';
    """
elif operation_type == "DELETE":
    # Delete the booking
    statement = f"""
    DELETE FROM datalink.lineagedemo.booking_status_info
    WHERE customer_id = '{customer_id}' AND vehicle_id = '{vehicle_id}';
    """
else:
    return f"Invalid operation type: {operation_type}. Use INSERT, UPDATE, or DELETE."

# Prepare slot update statements
slot_update_statements = []

# If inserting or updating to a new slot, mark it unavailable
if operation_type == "INSERT" or (operation_type == "UPDATE" and old_slot_id != slot_id):
    # Mark the new slot as booked
    slot_update_statements.append(f"""
    UPDATE datalink.lineagedemo.appointment_slots
    SET is_available = FALSE
    WHERE slot_id = '{slot_id}';
    """)
# If deleting or updating away from old slot, free up old slot
if operation_type == "DELETE" or (operation_type == "UPDATE" and old_slot_id != slot_id):
    # Free up the old slot if the slot has changed
    if old_slot_id:
        slot_update_statements.append(f"""
        UPDATE datalink.lineagedemo.appointment_slots
        SET is_available = TRUE
        WHERE slot_id = '{old_slot_id}';
        """)

# Execute the main booking operation
headers = {
    "Authorization": f"Bearer {bearer_token}",
    "Content-Type": "application/json"
}

payload = {
    "statement": statement,
    "wait_timeout": "50s",
    "warehouse_id": warehouse_id
}

response = requests.post(url, headers=headers, json=payload)

if response.status_code != 200:
    return f"Booking operation failed: {response.status_code} - {response.text}"

# Execute slot update SQLs
for slot_update in slot_update_statements:
    slot_payload = {
        "statement": slot_update,
        "wait_timeout": "50s",
        "warehouse_id": warehouse_id
    }
    slot_response = requests.post(url, headers=headers, json=slot_payload)
    if slot_response.status_code != 200:
        return f"Booking operation succeeded but slot update failed: {slot_response.status_code} - {slot_response.text}"

# Return success message
if operation_type == "INSERT":
    return f"Booking created. ID: {booking_id}, Status: CONFIRMED"
elif operation_type == "UPDATE":
    return "Booking updated successfully."
elif operation_type == "DELETE":
    return "Booking deleted successfully."
$$;

### GET_CUSTOMER_DETAILS

In [0]:
%sql
CREATE OR REPLACE FUNCTION datalink.lineagedemo.get_customer_details( Phone_no STRING)
RETURNS TABLE
(
  customer_id STRING,
  vehicle_id STRING, 
  customer_name STRING,
  phone_number STRING,
  email STRING,
  model STRING,
  plate_number STRING,
  year STRING
)
RETURN 
(
  -- Returns customer and their vehicle details based on the provided phone number
  SELECT 
    a.customer_id,vehicle_id,customer_name, phone_number, email,model,plate_number,year
  FROM 
    datalink.lineagedemo.customer a join vehicle_info b on a.customer_id=b.customer_id
  WHERE 
    phone_number = REGEXP_REPLACE(get_customer_details.Phone_no, '[^0-9]', '')
)